<a href="https://colab.research.google.com/github/visiont3lab/covid-mask-classifier/blob/master/utils/colab/RegressionPipelineCrossValidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* x1 x2 x3 x4 --> numeri --> (rosso,verde,blu)  --> Variabili categoriche , filtro
* Come scegliere il modello (RMSE (10) /MSQE) --> Test  --> Linear Regressison / Lasso / Elastic Search
* Pipeline Optimization (Automatic)

*


In [ ]:
from sklearn.datasets import load_boston, load_diabetes
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, TheilSenRegressor, RANSACRegressor, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures,StandardScaler
from joblib import dump, load
from sklearn.model_selection import KFold,cross_val_score

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/Frenz86/machine-learning-course/main/ShareRaceByCity.csv")
df.head()

'''
len(df["Geographic area"].unique())
len(df["City"].unique())

a = list(df["City"].unique())
p = [a[e] for e in [100,330,34,67,89]]

p = []
for e in [100,330,34,67,89]
    p.append(a[e])
    
p
'''

'\nlen(df["Geographic area"].unique())\nlen(df["City"].unique())\n\na = list(df["City"].unique())\np = [a[e] for e in [100,330,34,67,89]]\n\np = []\nfor e in [100,330,34,67,89]\n    p.append(a[e])\n    \np\n'

In [ ]:
# Filtro  Remove null values
df_clean = df[df["share_white"]!="(X)"]

# Cleaning and settings info type
dfnew_dict = {
 'Geographic area' : df_clean['Geographic area'],
  #'City': df_clean['City'],
 'share_white': np.array(df_clean['share_white'], dtype=np.float),
 'share_black' : np.array(df_clean['share_black'], dtype=np.float),
 'share_native_american' : np.array(df_clean['share_native_american'], dtype=np.float),
 'share_asian' : np.array(df_clean['share_asian'], dtype=np.float),
 'share_hispanic' : np.array(df_clean['share_hispanic'], dtype=np.float),
}
dfnew = pd.DataFrame(data=dfnew_dict)

ln = list(df["City"].unique())
city_names = [ln[e] for e in [100,330,34,67,89]]
#dfnew = dfnew[(dfnew["City"]==city_names[0]) | (dfnew["City"]==city_names[1]) | (dfnew["City"]==city_names[2])]
#dfnew = dfnew[(dfnew["City"]==city_names[0]) & (dfnew["share_hispanic"]>0.7)]

# Preprocessing
df_clean_mod = pd.get_dummies(dfnew, prefix_sep='-', drop_first=False)
display(df_clean_mod.head())

# Starting 
Y = df_clean_mod.pop("share_hispanic")
X = df_clean_mod.values
#Y = df_clean_mod["share_hispanic"].values

,share_white,share_black,share_native_american,share_asian,share_hispanic,Geographic area-AK,Geographic area-AL,Geographic area-AR,Geographic area-AZ,Geographic area-CA,Geographic area-CO,Geographic area-CT,Geographic area-DC,Geographic area-DE,Geographic area-FL,Geographic area-GA,Geographic area-HI,Geographic area-IA,Geographic area-ID,Geographic area-IL,Geographic area-IN,Geographic area-KS,Geographic area-KY,Geographic area-LA,Geographic area-MA,Geographic area-MD,Geographic area-ME,Geographic area-MI,Geographic area-MN,Geographic area-MO,Geographic area-MS,Geographic area-MT,Geographic area-NC,Geographic area-ND,Geographic area-NE,Geographic area-NH,Geographic area-NJ,Geographic area-NM,Geographic area-NV,Geographic area-NY,Geographic area-OH,Geographic area-OK,Geographic area-OR,Geographic area-PA,Geographic area-RI,Geographic area-SC,Geographic area-SD,Geographic area-TN,Geographic area-TX,Geographic area-UT,Geographic area-VA,Geographic area-VT,Geographic area-WA,Geographic area-WI,Geographic area-WV,Geographic area-WY
0,67.2,30.2,0.0,0.0,1.6,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,54.4,41.4,0.1,1.0,3.1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,52.3,44.9,0.5,0.3,2.3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,99.1,0.1,0.0,0.1,0.4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,13.2,86.5,0.0,0.0,0.3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Compariamo il modello
models = {}
models["Linear"]       = LinearRegression()
models["Lasso"]        = Lasso()
models["Ridge"]        = Ridge()
models["Theilsen"]     = TheilSenRegressor()
models["ElasticNet"]   = ElasticNet()
models["DecisionTree"]  = DecisionTreeRegressor()
models["RandomForest"]  = RandomForestRegressor()
folds   = 5 # 10 = 10%, 5 = 20% for testing
#metric  = "neg_mean_squared_error"
metric = "neg_root_mean_squared_error"
model_results = []
model_names   = []
for model_name in models:
    model   = models[model_name]
    pipe = Pipeline([
        ("std", StandardScaler())  ,          
        ("model", model)
    ])
    k_fold  = KFold(n_splits=folds, random_state=25,shuffle=True)
    results = cross_val_score(pipe, X, Y, cv=k_fold, scoring=metric)
    model_results.append(results)
    model_names.append(model_name)
    print("{}: {}, {}".format(model_name, np.round(results.mean(), 3), np.round(results.std(), 3)))


Linear: -4397284333.483, 8794568647.535
Lasso: -12.818, 0.219
Ridge: -9.803, 0.298
Theilsen: -10.968, 0.289
ElasticNet: -13.999, 0.215
DecisionTree: -9.994, 0.543


In [ ]:
fig = go.Figure()
for name,res in zip(model_names,model_results):    
    fig.add_trace(go.Box(y=res,name=name, boxpoints='all'))
fig.show()

In [ ]:
## Retraiing su tutti i dati con il modello scelto

In [ ]:
# Train Test
X_train, X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.35, random_state=26, shuffle=True) # Migliri

# Training X_train -------------------------
#reg = RandomForestRegressor()  # instanzia la classe
#reg = GradientBoostingRegressor()
#reg = LinearRegression()
#reg.fit(X_train,Y_train) # allenamento
reg = Pipeline([
    ("std", StandardScaler())  ,          
    #("poly",PolynomialFeatures(degree=3,interaction_only=False)),
	#("reg", Lasso())
 	#("reg", ElasticNet())
    ("reg", TheilSenRegressor())
    #("reg", LinearRegression())
])
reg.fit(X_train,Y_train)
#print(np.round(reg.named_steps["reg"].coef_,3),reg.named_steps["reg"].intercept_)
#---------------------------------
dump(reg, 'mypipeline.joblib')

regload = load("mypipeline.joblib")
# Training Error
Y_hat_train = regload.predict(X_train)
err_sklearn_rmse = np.sqrt(mean_squared_error(Y_train,Y_hat_train))
err_sklearn_mae = mean_absolute_error(Y_train,Y_hat_train)
err_mae = np.sum(np.abs(Y_hat_train -Y_train))/len(Y_hat_train) # mean absolute error ()
err_rmse = np.sqrt(np.sum(np.power(Y_hat_train -Y_train,2))/len(Y_hat_train)) # mean squared error ()
print("Training sklaern mse ", err_sklearn_rmse)
print("Training  err mae", np.round(err_mae,3))
print("Training err rmse ", np.round(err_rmse,3)) 

# Testing Error
Y_hat_test = regload.predict(X_test)
err_sklearn_rmse = np.sqrt(mean_squared_error(Y_test,Y_hat_test))
err_sklearn_mae = mean_absolute_error(Y_test,Y_hat_test)
err_mae = np.sum(np.abs(Y_hat_test -Y_test))/len(Y_hat_test) # mean absolute error ()
err_rmse = np.sqrt(np.sum(np.power(Y_hat_test -Y_test,2))/len(Y_hat_test)) # mean squared error ()

print("Test sklaern mse ", err_sklearn_rmse)
print("Test  err mae", np.round(err_mae,3))
print("Test err rmse ", np.round(err_rmse,3)) 


# Figure Test Results
xaxis = np.linspace(1,len(Y_train),len(Y_train),dtype=np.int)
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=xaxis, y=Y_train,mode='lines+markers', name="Y Train")
)
fig.add_trace(
    go.Scatter(x=xaxis, y=Y_hat_train,mode='lines+markers', name="Y Hat Train")
)
fig.update_layout(hovermode="x", title="Train Results", xaxis_title="Samples",yaxis_title="Price")
fig.write_html("Train.html")
fig.show()

# Figure Test Results
xaxis = np.linspace(1,len(Y_test),len(Y_test),dtype=np.int)
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=xaxis, y=Y_test,mode='lines+markers', name="Y Test")
)
fig.add_trace(
    go.Scatter(x=xaxis, y=Y_hat_test,mode='lines+markers', name="Y Hat Test")
)
fig.update_layout(hovermode="x", title="Test Results", xaxis_title="Samples",yaxis_title="Price")
fig.write_html("Test.html")
fig.show()


Training sklaern mse  11.11343831990247
Training  err mae 5.456
Training err rmse  11.113
Test sklaern mse  10.486631496466831
Test  err mae 5.211
Test err rmse  10.487


## Cross validation Spli example

In [6]:
import numpy as np
from sklearn.model_selection import KFold
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
Y = np.array([1, 2, 3, 4])

kf = KFold(n_splits=2,random_state=25,shuffle=True)
print("Number of folds:" , kf.get_n_splits(X))
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_Train, X_test = X[train_index], X[test_index]
    Y_Train, Y_test = Y[train_index], Y[test_index]
    print(X_Train,Y_Train)



Number of folds: 2
TRAIN: [0 2] TEST: [1 3]
[[1 2]
 [1 2]] [1 3]
TRAIN: [1 3] TEST: [0 2]
[[3 4]
 [3 4]] [2 4]
